In [1]:

import torch
import torch.nn as nn
import torchvision.transforms as transforms
import timm
import torchsummary
import imageio
import numpy as np
import matplotlib.pyplot as plt
import os
from torch.cuda.amp import GradScaler, autocast

from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split, KFold

import sys
sys.path.append('../DataLoader')

from dataloader_fits import SunImageDataset

from torch.func import stack_module_state
from torch.func import vmap

from lightning.fabric import Fabric

In [2]:
torch.set_float32_matmul_precision('medium')
fabric = Fabric(accelerator='cuda', devices=1, precision="bf16-mixed")
fabric.launch()
print(fabric.device)

Using bfloat16 Automatic Mixed Precision (AMP)


cuda:0


In [ ]:
# Hyper-parameters
input_size = 224*224
hidden_size = 166
# hidden_size = 1024
num_epochs = 9
batch_size = 2
learning_rate = 0.001
dropout = 0.5
lstm_layers = 2
# dropout = 0.6990787087509548

In [4]:
dataset = SunImageDataset(csv_file="D:\\New folder (2)\\dataset.csv", offset=0, transform=transforms.ToTensor())
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

# # Without Validation Set
# trainset, testset = torch.utils.data.Subset(dataset, range(train_size)), torch.utils.data.Subset(dataset, range(train_size, len(dataset)))
# trainloader = torch.utils.data.DataLoader(dataset=trainset, batch_size=batch_size, shuffle=True, num_workers=10)

# trainloader = fabric.setup_dataloaders(trainloader)

# With Validation Set
# Split dataset into training and test sets
train_indices, test_indices = train_test_split(range(len(dataset)), test_size=0.2, shuffle=False)

# Further split training set into training and validation sets
train_indices, val_indices = train_test_split(train_indices, test_size=0.25, shuffle=False)  # 0.25 x 0.8 = 0.2

trainset = torch.utils.data.Subset(dataset, train_indices)
valset = torch.utils.data.Subset(dataset, val_indices)
testset = torch.utils.data.Subset(dataset, test_indices)

trainloader = torch.utils.data.DataLoader(dataset=trainset, batch_size=batch_size, shuffle=True, num_workers=10)
valloader = torch.utils.data.DataLoader(dataset=valset, batch_size=batch_size, shuffle=False, num_workers=10)

trainloader = fabric.setup_dataloaders(trainloader)
valloader = fabric.setup_dataloaders(valloader)


# # Get a batch of training data
# dataiter = iter(trainloader)
# images, labels = next(dataiter)
# images = torch.stack(images)
# print(images.shape)
# print(labels.shape)

# print(images)
# print(labels)

In [5]:

class GmiSwinTransformer(nn.Module):
    def __init__(self, hidden_size: int):
        super(GmiSwinTransformer, self).__init__()
        
        # Batch normalization for 3 channels
        self.bn = nn.BatchNorm2d(3)
        
        # Initialize Swin Transformer
        self.pretrained_model = timm.create_model(
            'swin_base_patch4_window7_224',
            pretrained=True,
            num_classes=hidden_size
        )
        
        self.lstm = nn.LSTM(hidden_size, hidden_size * 2, lstm_layers, batch_first=True)
        
        # Fully connected layers
        self.fc = nn.Sequential(
            nn.LeakyReLU(),
            nn.Linear(hidden_size*2, hidden_size),
            nn.Dropout(p=dropout),  # Added dropout probability
            nn.LeakyReLU(),
            nn.Linear(hidden_size, 1),
            # nn.LeakyReLU()
            
        )
    
    def forward(self, images) -> torch.Tensor:
        """
        Batch should be in format:
        {
            'images': torch.FloatTensor((10, 1, 224, 224))
        }
        """
        
        batch_size = images.shape[0]
        
        images = images.reshape(-1, 1, 224, 224)
        images = torch.cat([images, images, images], dim=1)
        normalized_images = self.bn(images)
        features = self.pretrained_model(normalized_images)
        image_features = features.view(batch_size, 10, -1)
        # hidden_states = torch.zeros(lstm_layers, image_features.size(0), hidden_size * 2)
        # cell_states = torch.zeros(lstm_layers, image_features.size(0), hidden_size * 2)
        # image_features, _ = self.lstm(image_features, (hidden_states, cell_states))
        image_features, _ = self.lstm(image_features)

        image_features = image_features[:, -1, :]
        output = self.fc(image_features)
        return output

# Initialize model
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = GmiSwinTransformer(hidden_size=hidden_size).to(device)
model = GmiSwinTransformer(hidden_size=hidden_size)

# print(torchsummary.summary(model, (10, 1, 224, 224)))

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

model, optimizer = fabric.setup(model, optimizer)
model.train()

_FabricModule(
  (_forward_module): GmiSwinTransformer(
    (bn): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pretrained_model): SwinTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
        (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      )
      (layers): Sequential(
        (0): SwinTransformerStage(
          (downsample): Identity()
          (blocks): Sequential(
            (0): SwinTransformerBlock(
              (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
              (attn): WindowAttention(
                (qkv): Linear(in_features=128, out_features=384, bias=True)
                (attn_drop): Dropout(p=0.0, inplace=False)
                (proj): Linear(in_features=128, out_features=128, bias=True)
                (proj_drop): Dropout(p=0.0, inplace=False)
                (softmax): Softmax(dim=-1)
              )
              (drop_pa

In [6]:
torch.cuda.empty_cache()

In [7]:
# Training the model
n_total_steps = len(trainloader)
avg_train_loss_over_epochs = []
avg_val_loss_over_epochs = []

for epoch in range(num_epochs):
    train_losses = []
    val_losses = []
    
    # Training loop
    model.train()
    for i, (images, labels) in tqdm(enumerate(trainloader), desc="Training Progress", total=len(trainloader)):
        # Move images and labels to device
        images = images.float()
        # images = images.permute(1, 0, 2, 3, 4)  # Change shape to [5, 10, 1, 224, 224]
        labels = labels.float()

        # Forward pass with autograd
        outputs = model(images).squeeze(1)
        loss = criterion(outputs, labels)
        tqdm.write(f"Epoch: {epoch+1}, Index: {i}, Loss: {loss.item():.4f}")

        # Backward pass and optimization
        optimizer.zero_grad()
        fabric.backward(loss)
        optimizer.step()
        # Store the loss
        train_losses.append(loss.item())
    
    # Store the average training loss for this epoch
    avg_train_loss_over_epochs.append(sum(train_losses) / len(train_losses))
    
    # Validation loop
    model.eval()
    with torch.no_grad():
        for images, labels in tqdm(valloader, desc="Validation Progress"):
            images = images.float()
            # images = images.permute(1, 0, 2, 3, 4)
            labels = labels.float()
            outputs = model(images).squeeze(1)
            loss = criterion(outputs, labels)
            print("Validation Loss: ", loss.item())
            val_losses.append(loss.item())
    
    # Store the average validation loss for this epoch
    avg_val_loss_over_epochs.append(sum(val_losses) / len(val_losses))

# Plot loss over epochs
plt.figure()
plt.plot(range(1, num_epochs + 1), avg_train_loss_over_epochs, label='Average Training Loss', marker='o')
plt.plot(range(1, num_epochs + 1), avg_val_loss_over_epochs, label='Average Validation Loss', marker='o')
plt.xticks(range(1, num_epochs + 1))  # Ensure x-axis includes all epoch numbers
plt.xlabel('Epoch')
plt.ylabel('Average Loss')
plt.legend()
plt.title('Training and Validation Loss Over Epochs')
plt.show()

Training Progress:   0%|          | 0/600 [00:00<?, ?it/s]

Epoch: 1, Index: 0, Loss: 9.7317
Epoch: 1, Index: 1, Loss: 25.6552
Epoch: 1, Index: 2, Loss: 10.9183
Epoch: 1, Index: 3, Loss: 8.6302
Epoch: 1, Index: 4, Loss: 6.5508
Epoch: 1, Index: 5, Loss: 5.4101
Epoch: 1, Index: 6, Loss: 7.1116
Epoch: 1, Index: 7, Loss: 3.3374
Epoch: 1, Index: 8, Loss: 3.1706
Epoch: 1, Index: 9, Loss: 4.9486
Epoch: 1, Index: 10, Loss: 13.5012
Epoch: 1, Index: 11, Loss: 2.1972
Epoch: 1, Index: 12, Loss: 9.1906
Epoch: 1, Index: 13, Loss: 12.4777
Epoch: 1, Index: 14, Loss: 2.6289
Epoch: 1, Index: 15, Loss: 13.1806
Epoch: 1, Index: 16, Loss: 2.0725
Epoch: 1, Index: 17, Loss: 12.3634
Epoch: 1, Index: 18, Loss: 22.3300
Epoch: 1, Index: 19, Loss: 13.8697
Epoch: 1, Index: 20, Loss: 15.3398
Epoch: 1, Index: 21, Loss: 8.8045
Epoch: 1, Index: 22, Loss: 8.1408
Epoch: 1, Index: 23, Loss: 9.6191
Epoch: 1, Index: 24, Loss: 8.2896
Epoch: 1, Index: 25, Loss: 16.9198
Epoch: 1, Index: 26, Loss: 7.4047
Epoch: 1, Index: 27, Loss: 10.5271
Epoch: 1, Index: 28, Loss: 3.4120
Epoch: 1, Ind

Validation Progress:   0%|          | 0/200 [00:00<?, ?it/s]

Validation Loss:  0.4733283221721649
Validation Loss:  0.6690245866775513
Validation Loss:  0.3689376413822174
Validation Loss:  0.4482611417770386
Validation Loss:  0.2971828281879425
Validation Loss:  0.7862658500671387
Validation Loss:  0.8015549182891846
Validation Loss:  0.3658447265625
Validation Loss:  3.077359676361084
Validation Loss:  4.731789588928223
Validation Loss:  1.6868215799331665
Validation Loss:  1.2561430931091309
Validation Loss:  1.3669641017913818
Validation Loss:  1.7364501953125
Validation Loss:  1.6097043752670288
Validation Loss:  2.2677001953125
Validation Loss:  0.26086562871932983
Validation Loss:  1.456569790840149
Validation Loss:  1.256234884262085
Validation Loss:  1.3179799318313599
Validation Loss:  1.0747580528259277
Validation Loss:  0.15415562689304352
Validation Loss:  0.660400390625
Validation Loss:  2.7555627822875977
Validation Loss:  0.375244140625
Validation Loss:  3.7890427112579346
Validation Loss:  0.13445326685905457
Validation Loss:  0

Training Progress:   0%|          | 0/600 [00:00<?, ?it/s]

...

Validation Loss:  1.0508947372436523
Validation Loss:  1.9790542125701904
Validation Loss:  0.9643752574920654
Validation Loss:  0.20483437180519104
Validation Loss:  1.2571340799331665
Validation Loss:  1.4958391189575195
Validation Loss:  1.3516062498092651
Validation Loss:  1.5631103515625
Validation Loss:  2.145453453063965
Validation Loss:  4.874882698059082
Validation Loss:  0.9617921710014343
Validation Loss:  1.6329174041748047
Validation Loss:  0.5223703980445862
Validation Loss:  1.196533203125
Validation Loss:  1.403368353843689
Validation Loss:  2.393585205078125
Validation Loss:  0.4688698649406433
Validation Loss:  2.622938394546509
Validation Loss:  0.3505533039569855
Validation Loss:  2.014620542526245
Validation Loss:  2.0618247985839844
Validation Loss:  0.22825543582439423
Validation Loss:  0.376129150390625
Validation Loss:  0.18834514915943146
Validation Loss:  0.5030517578125
Validation Loss:  2.515554428100586
Validation Loss:  0.5869327187538147
Validation Loss:

Training Progress:   0%|          | 0/600 [00:00<?, ?it/s]

Epoch: 30, Index: 0, Loss: 0.0488
Epoch: 30, Index: 1, Loss: 0.0300
Epoch: 30, Index: 2, Loss: 0.3286
Epoch: 30, Index: 3, Loss: 0.2141
Epoch: 30, Index: 4, Loss: 0.0938
Epoch: 30, Index: 5, Loss: 0.0504
Epoch: 30, Index: 6, Loss: 0.0758
Epoch: 30, Index: 7, Loss: 0.3189
Epoch: 30, Index: 8, Loss: 0.1600
Epoch: 30, Index: 9, Loss: 0.0228
Epoch: 30, Index: 10, Loss: 0.9724
Epoch: 30, Index: 11, Loss: 0.1359
Epoch: 30, Index: 12, Loss: 0.1053
Epoch: 30, Index: 13, Loss: 0.2035
Epoch: 30, Index: 14, Loss: 1.7006
Epoch: 30, Index: 15, Loss: 0.0947
Epoch: 30, Index: 16, Loss: 0.3970
Epoch: 30, Index: 17, Loss: 0.1420
Epoch: 30, Index: 18, Loss: 0.0353
Epoch: 30, Index: 19, Loss: 0.4108
Epoch: 30, Index: 20, Loss: 0.0515
Epoch: 30, Index: 21, Loss: 0.0254
Epoch: 30, Index: 22, Loss: 0.0291
Epoch: 30, Index: 23, Loss: 0.0411
Epoch: 30, Index: 24, Loss: 0.1100
Epoch: 30, Index: 25, Loss: 0.0767
Epoch: 30, Index: 26, Loss: 0.0049
Epoch: 30, Index: 27, Loss: 0.0098
Epoch: 30, Index: 28, Loss: 0.

Validation Progress:   0%|          | 0/200 [00:00<?, ?it/s]

Validation Loss:  1.3228996992111206
Validation Loss:  1.4943110942840576
Validation Loss:  0.7776056528091431
Validation Loss:  0.1125931367278099
Validation Loss:  1.5946447849273682
Validation Loss:  1.4301047325134277
Validation Loss:  1.0795578956604004
Validation Loss:  1.675811767578125
Validation Loss:  1.9271869659423828
Validation Loss:  4.681402206420898
Validation Loss:  0.5548012256622314
Validation Loss:  1.9742212295532227
Validation Loss:  0.3098782002925873
Validation Loss:  1.1197509765625
Validation Loss:  1.60213303565979
Validation Loss:  1.966705322265625
Validation Loss:  0.48198485374450684
Validation Loss:  2.505422592163086
Validation Loss:  0.4656812250614166
Validation Loss:  1.7057543992996216
Validation Loss:  2.0810201168060303
Validation Loss:  0.0917268767952919
Validation Loss:  0.312042236328125
Validation Loss:  0.22652313113212585
Validation Loss:  0.6671142578125
Validation Loss:  2.698094606399536
Validation Loss:  0.20276574790477753
Validation L

In [8]:
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, root_mean_squared_error

# Test the model
model.eval()
testloader = torch.utils.data.DataLoader(dataset=testset, batch_size=batch_size, shuffle=False, num_workers=10)
testloader = fabric.setup_dataloaders(testloader)
test_losses = []
all_labels = []
all_outputs = []

with torch.no_grad():
    for images, labels in tqdm(testloader, desc="Testing Progress"):
        images = images.float()
        # images = images.permute(1, 0, 2, 3, 4)
        labels = labels.float()
        print("Label: ", labels)
        outputs = model(images).squeeze(1)
        print("Output: ", outputs)
        loss = criterion(outputs, labels)
        test_losses.append(loss.item())
        all_labels.extend(labels.cpu().numpy())
        all_outputs.extend(outputs.cpu().numpy())

avg_test_loss = sum(test_losses) / len(test_losses)
rmse = math.sqrt(avg_test_loss)
mse = mean_squared_error(all_labels, all_outputs)
mae = mean_absolute_error(all_labels, all_outputs)
r2 = r2_score(all_labels, all_outputs)

print(f'Average test loss: {avg_test_loss:.4f}')
print(f'RMSE: {rmse:.4f}')
print(f'MSE: {mse:.4f}')
print(f'MAE: {mae:.4f}')
print(f'R²: {r2:.4f}')

Testing Progress:   0%|          | 0/200 [00:00<?, ?it/s]

Label:  tensor([3.3330, 4.6670], device='cuda:0')
Output:  tensor([1.9688, 2.2656], device='cuda:0')
Label:  tensor([2.6670, 4.6670], device='cuda:0')
Output:  tensor([1.5000, 2.9062], device='cuda:0')
Label:  tensor([2.6670, 2.3330], device='cuda:0')
Output:  tensor([3.1719, 2.2188], device='cuda:0')
Label:  tensor([3.3330, 3.3330], device='cuda:0')
Output:  tensor([3.6562, 1.9375], device='cuda:0')
Label:  tensor([3.3330, 4.0000], device='cuda:0')
Output:  tensor([2.4375, 2.5469], device='cuda:0')
Label:  tensor([2.6670, 4.6670], device='cuda:0')
Output:  tensor([3.3438, 2.7969], device='cuda:0')
Label:  tensor([3.6670, 3.6670], device='cuda:0')
Output:  tensor([3.4062, 2.3281], device='cuda:0')
Label:  tensor([3.6670, 2.3330], device='cuda:0')
Output:  tensor([2.4062, 3.1094], device='cuda:0')
Label:  tensor([6., 3.], device='cuda:0')
Output:  tensor([2.7031, 2.5781], device='cuda:0')
Label:  tensor([3.0000, 2.3330], device='cuda:0')
Output:  tensor([2.7344, 2.1094], device='cuda:0'

In [9]:
plt.figure(figsize=(50, 6))

plt.plot(all_labels, '-', label='Actual Labels')
plt.plot(all_outputs, '--', label='Predicted Labels', alpha=0.7)

plt.xlabel('Sample Index')
plt.ylabel('Label Value')
plt.title('Overlay of Predicted and Actual Labels')
plt.legend()
plt.grid(True)
plt.show()
print(all_outputs)

[1.96875, 2.265625, 1.5, 2.90625, 3.171875, 2.21875, 3.65625, 1.9375, 2.4375, 2.546875, 3.34375, 2.796875, 3.40625, 2.328125, 2.40625, 3.109375, 2.703125, 2.578125, 2.734375, 2.109375, 1.5, 1.75, 1.984375, 1.625, 1.5703125, 2.734375, 3.328125, 3.0, 2.203125, 2.484375, 1.7265625, 1.2421875, 2.015625, 1.234375, 2.15625, 1.328125, 2.4375, 2.15625, 3.59375, 2.71875, 1.9921875, 2.5, 3.234375, 3.1875, 1.671875, 1.2421875, 1.7578125, 3.328125, 1.6484375, 3.15625, 2.625, 2.328125, 2.890625, 2.796875, 2.546875, 2.359375, 1.7734375, 1.390625, 2.5, 2.078125, 2.203125, 2.109375, 2.765625, 3.15625, 2.625, 2.8125, 2.921875, 1.359375, 3.234375, 3.6875, 2.375, 2.328125, 2.59375, 1.2421875, 2.28125, 2.71875, 3.28125, 2.8125, 2.6875, 3.5625, 3.40625, 2.78125, 2.03125, 2.65625, 1.46875, 1.953125, 3.8125, 3.640625, 1.9296875, 2.40625, 2.9375, 2.453125, 1.84375, 2.1875, 2.78125, 1.984375, 1.8125, 2.453125, 3.859375, 2.984375, 1.3515625, 2.125, 2.578125, 3.375, 3.78125, 3.328125, 1.953125, 2.359375, 2.04687